In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import urllib.request
import json

def get_elevation(lat, lng, sensor=False):
    """
    Returns the elevation of a specific location on earth using the Google
    Maps API.

    @param lat (float): The latitude of the location in degrees. Latitudes can
    take any value between -90 and 90.
    @param lng (float): The longitude of the location in degrees. Longitudes
    can take any value between -180 and 180.
    @param sensor (boolean): This parameter is required by the Google maps API
    and indicates whether the application that requests the elevation data is
    using a sensor (such as a GPS device). Default value is 'False'.

    @return: A tuple (elevation, lat, lng, status):
      * elevation (float): The requested elevation in meters. If the location is
        on the sea floor the returned elevation has a negative value.
      * lat, lng (float): The latitude and longitude of the location (for testing
        purposes: must be equal to the input values).
      * status (str): Error code:
        "OK": the API request was successful.
        "INVALID_REQUEST": the API request was malformed.
        "OVER_QUERY_LIMIT": the requester has exceeded quota.
        "REQUEST_DENIED": the API did not complete the request, likely because
        of an invalid 'sensor' parameter.
        "UNKNOWN_ERROR": other error
      * If the error code 'status' is not 'OK' then all other members of the
        returned tuple are set to 'None'.

    @note: More information about the Google elevation API and its usage limits
    can be found in https://developers.google.com/maps/documentation/elevation/.
    
    @example:
    >>> round(get_elevation(-38.407, -25.297)[0], 2) == -3843.86
    True
    >>> round(get_elevation(37.32522, -104.98470)[0], 2) == 2934.24    
    True
    """
    ## build the url for the API call
    ELEVATION_BASE_URL = 'https://api.open-elevation.com/api/v1/lookup'
    URL_PARAMS = "locations=%.7f,%.7f" % (lat, lng)
    url = ELEVATION_BASE_URL + "?" + URL_PARAMS

    ## make the call (ie. read the contents of the generated url) and decode the
    ## result (note: the result is in json format).
    with urllib.request.urlopen(url) as f:
        response = json.loads(f.read().decode())

        result = response["results"][0]
        elevation = float(result["elevation"])
        lat = float(result["location"]["latitude"])
        lng = float(result["location"]["longitude"])
        
    return (elevation, lat, lng)



In [29]:
get_elevation(46.739566, 11.959948)

KeyError: 'location'

In [ ]:
get_elevation(46.742419, 11.953511)

In [10]:
get_elevation(46.738837, 11.964165)

(2197.0, 46.738837, 11.964165)

In [11]:
import time
from osmread import parse_file, Node

In [12]:
%%time

filename = '../Data/Kronplatz_plus.osm'
nodes = []
for entity in parse_file(filename):
    if isinstance(entity, Node):
        #print(entity.id, entity.lon, entity.lat)
        elev = get_elevation(entity.lat, entity.lon)[0]
        node = {'id': entity.id, 'lat': entity.lat, 'lon': entity.lon, 'elevation': elev}
        nodes.append(node)
        time.sleep(.01)

print("Done")

Done
CPU times: user 49.1 s, sys: 2.6 s, total: 51.7 s
Wall time: 8min 14s


In [13]:
nodes

[{'elevation': 1772.0, 'id': 282865058, 'lat': 46.7485677, 'lon': 11.9840903},
 {'elevation': 1772.0, 'id': 282865059, 'lat': 46.7486928, 'lon': 11.9839817},
 {'elevation': 1772.0, 'id': 282865060, 'lat': 46.7488543, 'lon': 11.9839333},
 {'elevation': 1772.0, 'id': 282865061, 'lat': 46.7491409, 'lon': 11.9837597},
 {'elevation': 1772.0, 'id': 282865062, 'lat': 46.7494719, 'lon': 11.9836669},
 {'elevation': 1772.0, 'id': 282865063, 'lat': 46.749597, 'lon': 11.98357},
 {'elevation': 1772.0, 'id': 282865064, 'lat': 46.7497787, 'lon': 11.9834085},
 {'elevation': 1811.0, 'id': 282865065, 'lat': 46.7500855, 'lon': 11.9832753},
 {'elevation': 1185.0, 'id': 282865093, 'lat': 46.7449889, 'lon': 12.0097179},
 {'elevation': 1680.0, 'id': 282865094, 'lat': 46.7433392, 'lon': 11.9901255},
 {'elevation': 2274.0, 'id': 283551172, 'lat': 46.7379001, 'lon': 11.9584703},
 {'elevation': 2072.0, 'id': 283551174, 'lat': 46.73208, 'lon': 11.9615861},
 {'elevation': 1156.0, 'id': 283553882, 'lat': 46.7466875

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(nodes)

In [17]:
df

,elevation,id,lat,lon
0,1772.0,282865058,46.748568,11.984090
1,1772.0,282865059,46.748693,11.983982
2,1772.0,282865060,46.748854,11.983933
3,1772.0,282865061,46.749141,11.983760
4,1772.0,282865062,46.749472,11.983667
5,1772.0,282865063,46.749597,11.983570
6,1772.0,282865064,46.749779,11.983408
7,1811.0,282865065,46.750085,11.983275
8,1185.0,282865093,46.744989,12.009718
9,1680.0,282865094,46.743339,11.990125


In [20]:
df.T.to_json('nodes_elevation.json')

In [19]:
df.to_csv('nodes_elevation.csv')